In [1]:
# Setup

# Use a personal library (no sudo needed)
dir.create("~/Rlibs", recursive = TRUE, showWarnings = FALSE)
.libPaths(c("~/Rlibs", .libPaths()))

# Make sure CRAN is set (some HPC images don’t have it)
options(repos = c(CRAN = "https://cloud.r-project.org"))

# Install the missing CRAN dep
install.packages("S7")

# Install Bioconductor manager if needed
if (!requireNamespace("BiocManager", quietly = TRUE)) {
  install.packages("BiocManager")
}

# Install fgsea + BiocParallel from Bioconductor
BiocManager::install(c("fgsea", "BiocParallel"), ask = FALSE, update = TRUE)

# (Optional but often needed)
install.packages(c("readr","dplyr","tidyr","ggplot2"))

Installing package into ‘/mnt/home/bisholea/Rlibs’
(as ‘lib’ is unspecified)

Warning message in install.packages("S7"):
“installation of package ‘S7’ had non-zero exit status”
'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cloud.r-project.org

Bioconductor version 3.20 (BiocManager 1.30.26), R 4.4.1 (2024-06-14)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'fgsea' 'BiocParallel'”
Old packages: 'magrittr', 'Matrix', 'reticulate', 'SeuratObject', 'tibble',
  'utf8'

Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘magrittr’ had non-zero exit status”
Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘Matrix’ had non-zero exit status

In [3]:
# Get common genes between AE, NMF, and MSigDB for GSEA

suppressPackageStartupMessages({
  library(readr); library(dplyr); library(msigdbr); library(stringr)
})

base_dir <- "/mnt/projects/debruinz_project/bisholea/capstone/gsea"

# Load W (genes × factors)
ae_var  <- read_csv(file.path(base_dir, "ae_adata_var_metadata_unique.csv"),  show_col_types = FALSE)
ae_W    <- read_csv(file.path(base_dir, "ae_adata_var_embeddings_unique.csv"),show_col_types = FALSE)
nmf_var <- read_csv(file.path(base_dir, "als_nmf_var_metadata_unique.csv"),   show_col_types = FALSE)
nmf_W   <- read_csv(file.path(base_dir, "als_nmf_var_embeddings_unique.csv"), show_col_types = FALSE)

stopifnot(nrow(ae_var) == nrow(ae_W), nrow(nmf_var) == nrow(nmf_W))

# Gene symbols (uppercased for robust matching)
AE_GENES  <- toupper(str_trim(ae_var$feature_name))
NMF_GENES <- toupper(str_trim(nmf_var$feature_name))
MSIG      <- msigdbr(species = "Homo sapiens", collection = "C5")
MSIG_GENES <- unique(toupper(MSIG$gene_symbol))

cat("AE genes:", length(unique(AE_GENES)),
    "NMF genes:", length(unique(NMF_GENES)),
    "MSigDB genes:", length(MSIG_GENES), "\n")

UNIVERSE <- Reduce(intersect, list(unique(AE_GENES), unique(NMF_GENES), MSIG_GENES))
cat("Common gene universe size:", length(UNIVERSE), "genes\n")

# Filter rows in W to universe
ae_idx  <- AE_GENES  %in% UNIVERSE
nmf_idx <- NMF_GENES %in% UNIVERSE
ae_W_common  <- ae_W[ae_idx, , drop = FALSE]
nmf_W_common <- nmf_W[nmf_idx, , drop = FALSE]
ae_genes_common  <- AE_GENES[ae_idx]
nmf_genes_common <- NMF_GENES[nmf_idx]

# Restrict pathway list to universe (optional but tidy)
msig_list_common <- split(MSIG$gene_symbol, MSIG$gs_name)
msig_list_common <- lapply(msig_list_common, function(g) intersect(toupper(g), UNIVERSE))
msig_list_common <- msig_list_common[lengths(msig_list_common) > 0]

cat("Pathways with ≥1 gene in universe:", length(msig_list_common), "\n")

# Save everything for the analysis chunk
saveRDS(list(
  ae_W_common = ae_W_common,
  nmf_W_common = nmf_W_common,
  ae_genes_common = ae_genes_common,
  nmf_genes_common = nmf_genes_common,
  universe = UNIVERSE,
  msig_list_common = msig_list_common
), file.path(base_dir, "common_gene_preproc.rds"))

Warning message:
“package ‘msigdbr’ was built under R version 4.4.3”
Warning message:
“package ‘stringr’ was built under R version 4.4.3”


AE genes: 40508 NMF genes: 60664 MSigDB genes: 19549 
Common gene universe size: 19165 genes
Pathways with ≥1 gene in universe: 16228 


In [4]:
# Run GSEA for both AE and NMF on common genes

suppressPackageStartupMessages({
  library(dplyr); library(tidyr); library(fgsea); library(BiocParallel); library(readr)
})

base_dir <- "/mnt/projects/debruinz_project/bisholea/capstone/gsea"
pp <- readRDS(file.path(base_dir, "common_gene_preproc.rds"))

run_fgsea_on_W <- function(W, genes, pathways, prefix) {
  # W is genes×factors, `genes` are row-matched symbols (already in universe)
  stopifnot(nrow(W) == length(genes))
  coln <- colnames(W)
  bp <- BiocParallel::MulticoreParam(workers = 30)

  res <- lapply(seq_along(coln), function(i) {
    f <- coln[i]
    stats <- as.numeric(W[[i]])
    names(stats) <- genes
    stats <- sort(stats[stats != 0], decreasing = TRUE)  # drop zeros, rank desc
    if (!length(stats)) return(tibble::tibble(pathway=character(), NES=numeric(), padj=numeric(), factor=f))
    fg <- fgsea(pathways, stats, eps=0, scoreType="pos", minSize=5, maxSize=500, BPPARAM=bp)
    dplyr::transmute(fg, pathway, NES, padj, factor = f)
  })
  comb <- dplyr::bind_rows(res)

  nes  <- comb %>%
    tidyr::pivot_wider(id_cols = pathway, names_from = factor, values_from = NES) %>%
    dplyr::arrange(pathway)
  padj <- comb %>%
    tidyr::pivot_wider(id_cols = pathway, names_from = factor, values_from = padj) %>%
    dplyr::arrange(pathway)
  counts <- comb %>% mutate(sig = padj < 0.01) %>% count(factor, wt=sig, name="n_sig") %>% arrange(desc(n_sig))

  readr::write_csv(padj,  file.path(base_dir, sprintf("%s_padj_matrix_allFactors_common_genes.csv", prefix)))
  readr::write_csv(nes,   file.path(base_dir, sprintf("%s_NES_matrix_allFactors_common_genes.csv", prefix)))
  readr::write_csv(counts,file.path(base_dir, sprintf("%s_factor_gsea_counts_common_genes.csv", prefix)))

  message(prefix, ": factors=", ncol(W), ", factors with ≥1 sig=", sum(counts$n_sig>0),
          ", total sig pairs=", sum(counts$n_sig))
  invisible(list(combined=comb, counts=counts))
}

ae_out  <- run_fgsea_on_W(pp$ae_W_common,  pp$ae_genes_common,  pp$msig_list_common, "ae")
nmf_out <- run_fgsea_on_W(pp$nmf_W_common, pp$nmf_genes_common, pp$msig_list_common, "nmf")

Warning message:
“package ‘BiocParallel’ was built under R version 4.4.3”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.05% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
ae: factors=256, factors with ≥1 sig=44, total sig pairs=720

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (75.26% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (80.56% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (81.71% of the 

In [5]:
# Save as RDS for plotting 

saveRDS(list(
  ae_W_common      = pp$ae_W_common,
  nmf_W_common     = pp$nmf_W_common,
  ae_genes_common  = pp$ae_genes_common,
  nmf_genes_common = pp$nmf_genes_common,
  msig_list_common = pp$msig_list_common,
  ae_combined      = ae_out$combined,  # optional but useful
  nmf_combined     = nmf_out$combined, # optional but useful
  minSize          = 5,                 # record params used
  alpha            = 0.01,
  ae_counts        = ae_out$counts,   # per-factor n_sig for AE
  nmf_counts       = nmf_out$counts   # per-factor n_sig for NMF
), file.path(base_dir, "gsea_common_for_plots.rds"))



In [6]:
suppressPackageStartupMessages({
  library(readr); library(dplyr); library(tidyr); library(stringr)
})

base_dir <- "/mnt/projects/debruinz_project/bisholea/capstone/gsea"

summarize_model <- function(model) {
  nes_file  <- file.path(base_dir, sprintf("%s_NES_matrix_allFactors_common_genes.csv",  model))
  padj_file <- file.path(base_dir, sprintf("%s_padj_matrix_allFactors_common_genes.csv", model))
  fdr <- 0.05

  nes_mat  <- readr::read_csv(nes_file,  show_col_types = FALSE)
  padj_mat <- readr::read_csv(padj_file, show_col_types = FALSE)

  # Ensure 'pathway' exists and column names are unique/sane
  stopifnot("pathway" %in% names(nes_mat), "pathway" %in% names(padj_mat))
  names(nes_mat)  <- make.unique(names(nes_mat))
  names(padj_mat) <- make.unique(names(padj_mat))

  # Only use columns that are in BOTH matrices (excluding 'pathway')
  factor_cols <- setdiff(intersect(names(nes_mat), names(padj_mat)), "pathway")
  if (!length(factor_cols)) stop("No overlapping factor columns between NES and padj matrices.")

  # Long forms for the SAME set of factors, then deduplicate (pathway, factor)
  nes_long <- nes_mat |>
    tidyr::pivot_longer(all_of(factor_cols), names_to = "factor", values_to = "NES") |>
    dplyr::group_by(pathway, factor) |>
    dplyr::summarise(NES = suppressWarnings(mean(NES, na.rm = TRUE)), .groups = "drop")

  padj_long <- padj_mat |>
    tidyr::pivot_longer(all_of(factor_cols), names_to = "factor", values_to = "padj") |>
    dplyr::group_by(pathway, factor) |>
    dplyr::summarise(padj = suppressWarnings(min(padj, na.rm = TRUE)), .groups = "drop")

  # Safe one-to-one join now
  res <- nes_long |>
    dplyr::left_join(padj_long, by = c("pathway", "factor")) |>
    dplyr::mutate(
      tested = !is.na(padj),
      sig    = tested & (padj < fdr)
    )

  safe_which_min <- function(x) {
    if (length(x) == 0L || all(is.na(x))) return(NA_integer_)
    which.min(replace(x, is.na(x), Inf))
  }

  factor_counts <- res |>
    dplyr::group_by(factor) |>
    dplyr::summarise(
      n_tested     = sum(tested, na.rm = TRUE),
      n_sig        = sum(sig,     na.rm = TRUE),
      frac_sig     = ifelse(n_tested > 0, n_sig / n_tested, NA_real_),
      best_idx     = safe_which_min(padj),
      best_padj    = suppressWarnings(min(padj, na.rm = TRUE)),
      best_pathway = if (!is.na(best_idx)) pathway[best_idx] else NA_character_,
      best_NES     = if (!is.na(best_idx)) NES[best_idx]     else NA_real_,
      .groups = "drop"
    ) |>
    dplyr::arrange(dplyr::desc(n_sig)) |>
    dplyr::select(-best_idx)

  cat("\n=== GSEA Summary (COMMON GENES, FDR < 0.05) ===\n")
  cat("Model:", toupper(model), "\n")
  cat("Factors:", nrow(factor_counts), "\n")
  cat("Factors with ≥1 enriched GO term:", sum(factor_counts$n_sig > 0, na.rm = TRUE), "\n")
  cat("Total enriched factor–pathway pairs:", sum(factor_counts$n_sig, na.rm = TRUE), "\n\n")

  stats <- factor_counts$n_sig
  print(tibble::tibble(
    mean   = mean(stats), median = median(stats), sd  = sd(stats),
    min    = min(stats),  q1     = unname(quantile(stats, 0.25)),
    q3     = unname(quantile(stats, 0.75)), max = max(stats)
  ))
  cat("\nTop 10 factors by # enriched GO terms:\n")
  print(dplyr::slice_head(factor_counts, n = 10))

  invisible(factor_counts)
}

summarize_model("ae")
summarize_model("nmf")



=== GSEA Summary (COMMON GENES, FDR < 0.05) ===
Model: AE 
Factors: 256 
Factors with ≥1 enriched GO term: 83 
Total enriched factor–pathway pairs: 1740 

# A tibble: 1 × 7
   mean median    sd   min    q1    q3   max
  <dbl>  <dbl> <dbl> <int> <dbl> <dbl> <int>
1  6.80      0  30.4     0     0  1.25   375

Top 10 factors by # enriched GO terms:
# A tibble: 10 × 7
   factor n_tested n_sig frac_sig best_padj best_pathway                best_NES
   <chr>     <int> <int>    <dbl>     <dbl> <chr>                          <dbl>
 1 F179      11494   375  0.0326   1.17e- 8 HP_MITOCHONDRIAL_MYOPATHY       1.55
 2 F102      11494   213  0.0185   6.78e- 3 GOBP_REGULATION_OF_NUCLEOC…     1.53
 3 F129      11494   145  0.0126   1.94e-21 GOMF_STRUCTURAL_MOLECULE_A…     1.13
 4 F198      11494    90  0.00783  6.74e-11 GOBP_ATP_BIOSYNTHETIC_PROC…     1.41
 5 F134      11494    86  0.00748  1.02e- 8 GOCC_SYNAPSE                    1.15
 6 F64       11494    74  0.00644  1.93e- 6 HP_ABNORMALITY_OF_VIS

In [8]:
library(readr); library(dplyr); library(tidyr); library(tibble)

ae_path  <- file.path(base_dir, "ae_padj_matrix_allFactors_common_genes.csv")
nmf_path <- file.path(base_dir, "nmf_padj_matrix_allFactors_common_genes.csv")

# Show just the column names
ae_cols  <- names(readr::read_csv(ae_path, n_max = 0, show_col_types = FALSE))
nmf_cols <- names(readr::read_csv(nmf_path, n_max = 0, show_col_types = FALSE))
ae_cols; nmf_cols

# Print first few rows (as a tibble) to see what's actually in there
readr::read_csv(ae_path, n_max = 5, show_col_types = FALSE)
readr::read_csv(nmf_path, n_max = 5, show_col_types = FALSE)


[1] "pathway" "F1"      "F2"      "F3"      "F4"      "F5"      "F6"     
  [8] "F7"      "F8"      "F9"      "F10"     "F11"     "F12"     "F13"    
 [15] "F14"     "F15"     "F16"     "F17"     "F18"     "F19"     "F20"    
 [22] "F21"     "F22"     "F23"     "F24"     "F25"     "F26"     "F27"    
 [29] "F28"     "F29"     "F30"     "F31"     "F32"     "F33"     "F34"    
 [36] "F35"     "F36"     "F37"     "F38"     "F39"     "F40"     "F41"    
 [43] "F42"     "F43"     "F44"     "F45"     "F46"     "F47"     "F48"    
 [50] "F49"     "F50"     "F51"     "F52"     "F53"     "F54"     "F55"    
 [57] "F56"     "F57"     "F58"     "F59"     "F60"     "F61"     "F62"    
 [64] "F63"     "F64"     "F65"     "F66"     "F67"     "F68"     "F69"    
 [71] "F70"     "F71"     "F72"     "F73"     "F74"     "F75"     "F76"    
 [78] "F77"     "F78"     "F79"     "F80"     "F81"     "F82"     "F83"    
 [85] "F84"     "F85"     "F86"     "F87"     "F88"     "F89"     "F90"    
 [92] "F91"     "F92"     "F93"     "F94"     "F95"     "F96"     "F97"    
 [99] "F98"     "F99"     "F100"    "F101"    "F102"    "F103"    "F104"   
[106] "F105"    "F106"    "F107"    "F108"    "F109"    "F110"    "F111"   
[113] "F112"    "F113"    "F114"    "F115"    "F116"    "F117"    "F118"   
[120] "F119"    "F120"    "F121"    "F122"    "F123"    "F124"    "F125"   
[127] "F126"    "F127"    "F128"    "F129"    "F130"    "F131"    "F132"   
[134] "F133"    "F134"    "F135"    "F136"    "F137"    "F138"    "F139"   
[141] "F140"    "F141"    "F142"    "F143"    "F144"    "F145"    "F146"   
[148] "F147"    "F148"    "F149"    "F150"    "F151"    "F152"    "F153"   
[155] "F154"    "F155"    "F156"    "F157"    "F158"    "F159"    "F160"   
[162] "F161"    "F162"    "F163"    "F164"    "F165"    "F166"    "F167"   
[169] "F168"    "F169"    "F170"    "F171"    "F172"    "F173"    "F174"   
[176] "F175"    "F176"    "F177"    "F178"    "F179"    "F180"    "F181"   
[183] "F182"    "F183"    "F184"    "F185"    "F186"    "F187"    "F188"   
[190] "F189"    "F190"    "F191"    "F192"    "F193"    "F194"    "F195"   
[197] "F196"    "F197"    "F198"    "F199"    "F200"    "F201"    "F202"   
[204] "F203"    "F204"    "F205"    "F206"    "F207"    "F208"    "F209"   
[211] "F210"    "F211"    "F212"    "F213"    "F214"    "F215"    "F216"   
[218] "F217"    "F218"    "F219"    "F220"    "F221"    "F222"    "F223"   
[225] "F224"    "F225"    "F226"    "F227"    "F228"    "F229"    "F230"   
[232] "F231"    "F232"    "F233"    "F234"    "F235"    "F236"    "F237"   
[239] "F238"    "F239"    "F240"    "F241"    "F242"    "F243"    "F244"   
[246] "F245"    "F246"    "F247"    "F248"    "F249"    "F250"    "F251"   
[253] "F252"    "F253"    "F254"    "F255"    "F256"

[1] "pathway" "F1"      "F2"      "F3"      "F4"      "F5"      "F6"     
  [8] "F7"      "F8"      "F9"      "F10"     "F11"     "F12"     "F13"    
 [15] "F14"     "F15"     "F16"     "F17"     "F18"     "F19"     "F20"    
 [22] "F21"     "F22"     "F23"     "F24"     "F25"     "F26"     "F27"    
 [29] "F28"     "F29"     "F30"     "F31"     "F32"     "F33"     "F34"    
 [36] "F35"     "F36"     "F37"     "F38"     "F39"     "F40"     "F41"    
 [43] "F42"     "F43"     "F44"     "F45"     "F46"     "F47"     "F48"    
 [50] "F49"     "F50"     "F51"     "F52"     "F53"     "F54"     "F55"    
 [57] "F56"     "F57"     "F58"     "F59"     "F60"     "F61"     "F62"    
 [64] "F63"     "F64"     "F65"     "F66"     "F67"     "F68"     "F69"    
 [71] "F70"     "F71"     "F72"     "F73"     "F74"     "F75"     "F76"    
 [78] "F77"     "F78"     "F79"     "F80"     "F81"     "F82"     "F83"    
 [85] "F84"     "F85"     "F86"     "F87"     "F88"     "F89"     "F90"    
 [92] "F91"     "F92"     "F93"     "F94"     "F95"     "F96"     "F97"    
 [99] "F98"     "F99"     "F100"    "F101"    "F102"    "F103"    "F104"   
[106] "F105"    "F106"    "F107"    "F108"    "F109"    "F110"    "F111"   
[113] "F112"    "F113"    "F114"    "F115"    "F116"    "F117"    "F118"   
[120] "F119"    "F120"    "F121"    "F122"    "F123"    "F124"    "F125"   
[127] "F126"    "F127"    "F128"    "F129"    "F130"    "F131"    "F132"   
[134] "F133"    "F134"    "F135"    "F136"    "F137"    "F138"    "F139"   
[141] "F140"    "F141"    "F142"    "F143"    "F144"    "F145"    "F146"   
[148] "F147"    "F148"    "F149"    "F150"    "F151"    "F152"    "F153"   
[155] "F154"    "F155"    "F156"    "F157"    "F158"    "F159"    "F160"   
[162] "F161"    "F162"    "F163"    "F164"    "F165"    "F166"    "F167"   
[169] "F168"    "F169"    "F170"    "F171"    "F172"    "F173"    "F174"   
[176] "F175"    "F176"    "F177"    "F178"    "F179"    "F180"    "F181"   
[183] "F182"    "F183"    "F184"    "F185"    "F186"    "F187"    "F188"   
[190] "F189"    "F190"    "F191"    "F192"    "F193"    "F194"    "F195"   
[197] "F196"    "F197"    "F198"    "F199"    "F200"

pathway,F1,F2,F3,F4,F5,F6,F7,F8,F9,⋯,F247,F248,F249,F250,F251,F252,F253,F254,F255,F256
<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<lgl>,<lgl>,⋯,<dbl>,<lgl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
GOBP_2FE_2S_CLUSTER_ASSEMBLY,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GOBP_2_OXOGLUTARATE_METABOLIC_PROCESS,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GOBP_3_UTR_MEDIATED_MRNA_DESTABILIZATION,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GOBP_3_UTR_MEDIATED_MRNA_STABILIZATION,NA,NA,NA,NA,NA,NA,0.9877996,NA,NA,⋯,1,NA,0.8823497,NA,NA,NA,NA,NA,NA,NA
GOBP_5S_CLASS_RRNA_TRANSCRIPTION_BY_RNA_POLYMERASE_III,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


pathway,F1,F2,F3,F4,F5,F6,F7,F8,F9,⋯,F191,F192,F193,F194,F195,F196,F197,F198,F199,F200
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,<dbl>,<lgl>
GOBP_10_FORMYLTETRAHYDROFOLATE_METABOLIC_PROCESS,NA,0.9663585,0.9622251,NA,NA,NA,NA,NA,NA,⋯,NA,1.0000000,NA,NA,NA,NA,NA,NA,1,NA
GOBP_2FE_2S_CLUSTER_ASSEMBLY,NA,0.7725215,NA,0.9460936,0.9905548,0.9136583,0.9321511,NA,NA,⋯,NA,0.9435082,NA,0.8573874,1.0000000,NA,NA,NA,1,NA
GOBP_2_OXOGLUTARATE_METABOLIC_PROCESS,0.8419336,0.9404722,0.9872835,0.9807538,0.8310180,0.9883880,0.8657225,NA,NA,⋯,NA,0.9632641,1.0000000,1.0000000,NA,NA,NA,NA,1,NA
GOBP_3_UTR_MEDIATED_MRNA_DESTABILIZATION,0.4827329,0.2491901,0.4266477,0.6491670,0.4422332,NA,0.9722695,0.7634538,0.3836789,⋯,NA,0.7329049,1.0000000,NA,0.9907967,NA,NA,0.6668559,1,NA
GOBP_3_UTR_MEDIATED_MRNA_STABILIZATION,0.7349244,0.9595890,0.8395008,0.2394132,0.5321413,1.0000000,1.0000000,0.8650736,0.8102550,⋯,NA,0.8007400,0.9799712,0.7996489,1.0000000,NA,NA,NA,1,NA
